instala con CPU en vez de GPU y lo instalamos sin CUDA
pip install torch torchvision torchaudio


## libraries 📚

In [3]:
import fitz  
import re
import spacy
import json
from transformers import LayoutLMTokenizer, LayoutLMForTokenClassification
import torch

In [4]:
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cpu
False


## model 🤖

In [5]:
# Cargar modelo de spaCy
nlp = spacy.load("es_core_news_sm")

# Cargar modelo de LayoutLM
tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")
model = LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased")

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# -------------------------Functions--------------------------------------

### Function with regex 📌

In [6]:
def extract_with_regex(pattern, text):
    """Extrae información usando una expresión regular."""
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1) if match else "No encontrado"

### Function to extract text of pdf 📌

In [7]:

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

### Funtion clean text 📌

In [8]:
def limpiar_texto(texto):
    """Normaliza y limpia el texto para mejorar la extracción de metadatos."""
    texto = re.sub(r'\s+', ' ', texto)  # Remueve espacios extra
    texto = re.sub(r'[^\x20-\x7E\xC0-\xFF]', '', texto)  # Remueve caracteres especiales
    return texto.strip()

### Function read a pdf 📌

In [9]:
def leer_pdf(ruta_pdf):
    """Lee un archivo PDF y devuelve su contenido en texto limpio."""
    documento = fitz.open(ruta_pdf)
    texto = ""
    for pagina in documento:
        texto += pagina.get_text("text") + "\n"
    return limpiar_texto(texto)

### Function from number to whole number 📌

In [10]:
def whole_number(romano):
    valores = {'M': 1000, 'CM': 900, 'D': 500, 'CD': 400, 'C': 100, 'XC': 90,
            'L': 50, 'XL': 40, 'X': 10, 'IX': 9, 'V': 5, 'IV': 4, 'I': 1}
    entero = 0
    i = 0
    while i < len(romano):
        if i + 1 < len(romano) and romano[i:i+2] in valores:
            entero += valores[romano[i:i+2]]
            i += 2
        else:
            entero += valores[romano[i]]
            i += 1
    return entero

### Delete spaces in titles 📌

In [11]:
def normalizar_espacios_titulo(texto):
    """
    Corrige textos OCR donde hay espacios innecesarios entre letras en frases clave.
    """
    frases_clave = [
        "SENADO DE LA REPÚBLICA",
        "CÁMARA DE REPRESENTANTES",
        "PROYECTOS DE LEY"
    ]

    for frase in frases_clave:
        # change the frase
        frase_regex = r'\s*'.join(frase)  # change in "S\s*E\s*N\s*A\s*D\s*O\s* ..."
        texto = re.sub(frase_regex, frase, texto, flags=re.IGNORECASE)  #remplaze

    return texto

### function to clasification of documents

# ---------------------------------------------------------------

### Function extract_metadata 📚

In [12]:

def extract_metadata(text):
    metadata = {}
    
    texto_limpio = limpiar_texto(text)
    texto_limpio = normalizar_espacios_titulo(texto_limpio)

    metadata["titulo"] = "Gaceta del Congreso"

    # Entidades participantes
    entidades_match = re.search(r"(SENADO Y CÁMARA|SENADO|CÁMARA)", text, re.IGNORECASE)
    metadata["entidades"] = entidades_match.group(0) if entidades_match else "Desconocido"

    # ISSN
    # issn_match = re.search(r"ISSN\s?(\d{4}-\d{4})", text)
    # metadata["issn"] = issn_match.group(1) if issn_match else "No encontrado"
    metadata["issn"] = extract_with_regex(r"I\s*S\s*S\s*N\s*(0\s*1\s*2\s*3\s*-\s*9\s*0\s*6\s*6)", text).replace(" ","")
    
    # Año y número de publicación
    patron_anio_numero = r"AÑO\s+([MDCLXVI]+)\s*-\s*N[°º]\s*(\d+)"
    coincidencia_anio_numero = re.search(patron_anio_numero, text, re.IGNORECASE)
    if coincidencia_anio_numero:
        anio_romano = coincidencia_anio_numero.group(1).strip()
        numero_publicacion = coincidencia_anio_numero.group(2).strip()

        metadata["anio_romano"] = anio_romano
        metadata["anio"] = whole_number(anio_romano)  #change value
        metadata["numero_publicacion"] = int(numero_publicacion)
    else:
        metadata["anio_romano"] = "No encontrado"
        metadata["anio"] = "No encontrado"
        metadata["numero_publicacion"] = "No encontrado"

    # Fecha
    #fecha_match = re.search(r"(\w+,\s\d+\sde\s\w+\sde\s\d{4})", text)
    #metadata["fecha"] = fecha_match.group(0) if fecha_match else "No encontrada"
    fecha_match = re.search(r"(?i)(lunes|martes|miércoles|jueves|viernes|sábado|domingo),\s*(\d{1,2})\s+de\s+(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+de\s+(\d{4})", text)

    if fecha_match:
        meses = {
            "enero": "01", "febrero": "02", "marzo": "03", "abril": "04", "mayo": "05", "junio": "06",
            "julio": "07", "agosto": "08", "septiembre": "09", "octubre": "10", "noviembre": "11", "diciembre": "12"
        }
        dia = int(fecha_match.group(2))
        mes = meses[fecha_match.group(3).lower()]
        anio = int(fecha_match.group(4))
        metadata["fecha"] = f"{dia:02d}/{mes}/{anio}"
    else:
        metadata["fecha"] = "No encontrada"
    
    # Rama legislativa
    metadata["rama_legislativa"] = "RAMA LEGISLATIVA DEL PODER PÚBLICO"

    # Buscar la sección de directores -----------------------------
    # Extraer solo el nombre del director antes del título
    senado_match = re.search(r"\n([A-ZÁÉÍÓÚÑ]+\s+[A-ZÁÉÍÓÚÑ]+\s+[A-ZÁÉÍÓÚÑ]+)\nSECRETARIO\s+GENERAL\s+DEL\s+SENADO", text)
    camara_match = re.search(r"\n([A-ZÁÉÍÓÚÑ]+\s+[A-ZÁÉÍÓÚÑ]+\s+[A-ZÁÉÍÓÚÑ]+)\nSECRETARIO\s+GENERAL\s+DE\s+LA\s+CÁMARA", text)

    # Función para limpiar nombres (remueve espacios extra y caracteres erróneos)
    def clean_name(name):
        if name:
            name = name.strip()  # Eliminar espacios en los extremos
            name = re.sub(r'\s+', ' ', name)  # Reemplazar múltiples espacios por uno solo
            return name
        return "No encontrado"

    metadata["directores"] = {
        "director_senado": clean_name(senado_match.group(1)) if senado_match else "No encontrado",
        "director_camara": clean_name(camara_match.group(1)) if camara_match else "No encontrado"
    }
    
    # Extracción del título (Ejemplo: "CAMARA DE REPRESENTANTES", "PROYECTOS DE LEY", "SENADO DE LA REPÚBLICA")
    pattern_title = r"(?i)\b(SENADO\s+DE\s+LA\s+REPÚBLICA|CÁMARA\s+DE\s+REPRESENTANTES|PROYECTOS\s+DE\s+LEY)\b"
    match_title = re.search(pattern_title, texto_limpio)
    
    metadata["titulo"] = match_title.group(1) if match_title else "No encontrado"

    # Tipo de documento
    tipo_doc_match = re.search(r"(PONENCIAS|ACTAS|INFORMES|PROYECTOS DE LEY)", text)
    metadata["tipo_documento"] = tipo_doc_match.group(0) if tipo_doc_match else "No identificado"

    # Subtítulo
    subtitulo_match = re.search(r"([A-Z\s]+)\n+\"(.+?)\"", text)
    metadata["subtitulo"] = subtitulo_match.group(1).strip() if subtitulo_match else "No identificado"
    metadata["subsubtitulo"] = subtitulo_match.group(2).strip() if subtitulo_match else "No identificado"

    return metadata

### Principall function to extract metadata of PDF 🐦‍⬛

In [13]:
def extract_metadata_from_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    metadata = extract_metadata(text)

    # Convertir a JSON
    metadata_json = json.dumps(metadata, indent=4, ensure_ascii=False)
    return metadata_json

# ---------use🤖----------

In [14]:
pdf_path = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328_XXV_110_64_removed.pdf"
metadata_json = extract_metadata_from_pdf(pdf_path)
print(metadata_json)

FileNotFoundError: no such file: 'C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328_XXV_110_64_removed.pdf'

# saber el OCR ⌨️

In [ ]:
text = extract_text_from_pdf(pdf_path)
print(text) 

GACETA DEL CONGRESO  110  
Lunes, 28 de marzo de 2016 
Página 1
C O M E N TA R I O S
(Artículo 36,  Ley 5ª de 1992)
IMPRENTA   NACIONAL   DE   COLOMBIA
www.imprenta.gov.co
SENADO Y CÁMARA
AÑO XXV - Nº 110 
    Bogotá, D. C., lunes, 28 de marzo de 2016 
 EDICIÓN  DE  64  PÁGINAS
DIRECTORES: 
REPÚBLICA   DE   COLOMBIA
RAMA  LEGISLATIVA  DEL  PODER  PÚBLICO
S E N A D O   D E   L A   R E P Ú B L I C A
JORGE HUMBERTO MANTILLA SERRANO
SECRETARIO  GENERAL  DE  LA  CÁMARA
www.camara.gov.co 
GREGORIO ELJACH PACHECO
SECRETARIO  GENERAL  DEL  SENADO
www.secretariasenado.gov.co 
G A C E T A   D E L   C O N G R E S O
I S S N  0 1 2 3  -  9 0 6 6
COMENTARIOS AL PROYECTO DE LEY NÚMERO 97 DE 2015 SENADO
“por la cual se prohíbe la producción, comercialización, exportación, importación  
y distribución de cualquier variedad de asbesto en Colombia”. 
Mg
Mg
RESEÑA
respecto a la diferencia entre el crisotilo  
y los asbestos anﬁboles
“El asbesto” no es en si un mineral. Es un término colectivo que se da a 

### saber si reconoce un grupo con expression regular

In [ ]:
# Imprimir la sección de directores para ver su estructura real
directores_match = re.search(r"DIRECTORES?:(.*?)\n(?:SECREATRIO\s+GENERAL\s+DE\s+LA\+NACION|www|SECRETARIO\s+GENERAL\s+DE\s+LA\s+CÁMARA|)", text, re.DOTALL)

print(directores_match.group(1) if directores_match else "No se encontró la sección de directores")
